In [1]:
import subprocess
import glob
import os
from os.path import exists
import re
#from multiprocessing.dummy import Pool as ThreadPool
from multiprocessing import Pool

#### Build HGFM index with transcripts

In [2]:
! hisat2_extract_splice_sites.py /media/HDD2/Genomes/Ath_Ensembl56/Arabidopsis_thaliana.TAIR10.56.gtf > \
    /media/HDD2/Genomes/Ath_Ensembl56/Arabidopsis_thaliana.TAIR10.56.ss

In [3]:
! hisat2_extract_exons.py /media/HDD2/Genomes/Ath_Ensembl56/Arabidopsis_thaliana.TAIR10.56.gtf > \
    /media/HDD2/Genomes/Ath_Ensembl56/Arabidopsis_thaliana.TAIR10.56.exon

In [4]:
! hisat2-build -p 40 --exon  /media/HDD2/Genomes/Ath_Ensembl56/Arabidopsis_thaliana.TAIR10.56.exon \
    --ss /media/HDD2/Genomes/Ath_Ensembl56/Arabidopsis_thaliana.TAIR10.56.ss \
    /media/HDD2/Genomes/Ath_Ensembl56/Arabidopsis_thaliana.TAIR10.dna.toplevel.fa \
    /media/HDD2/Genomes/Ath_Ensembl56/HGFM_  # the prefix of index

Settings:
  Output files: "/media/HDD2/Genomes/Ath_Ensembl56/HGFM_.*.ht2"
  Line rate: 7 (line is 128 bytes)
  Lines per side: 1 (side is 128 bytes)
  Offset rate: 4 (one in 16)
  FTable chars: 10
  Strings: unpacked
  Local offset rate: 3 (one in 8)
  Local fTable chars: 6
  Local sequence length: 57344
  Local sequence overlap between two consecutive indexes: 1024
  Endianness: little
  Actual local endianness: little
  Sanity checking: disabled
  Assertions: disabled
  Random seed: 0
  Sizeofs: void*:8, int:4, long:8, size_t:8
Input files DNA, FASTA:
  /media/HDD2/Genomes/Ath_Ensembl56/Arabidopsis_thaliana.TAIR10.dna.toplevel.fa
Reading reference sizes
  Time reading reference sizes: 00:00:00
Calculating joined length
Writing header
Reserving space for joined string
Joining reference sequences
  Time to join reference sequences: 00:00:00
  Time to read SNPs and splice sites: 00:00:00
Generation 0 (119613280 -> 119613280 nodes, 0 ranks)
COUNTED NEW NODES: 0
COUNTED TEMP NODES: 0
RESI

In [3]:
fastq_dir = '/media/HDD3/JC14/'

In [4]:
list_of_fq = {}
for (dirpath, dirnames, filenames) in os.walk(fastq_dir):
    for filename in filenames:
        if filename.endswith('fq.gz'):
            list_of_fq[filename] = os.sep.join([dirpath, filename])

list_of_fq

{'CJ-2-LFD4716_L2_2.fq.gz': '/media/HDD3/JC14/N2304889_30-852716646_RNA_2023-04-21/raw_data/CJ-2-LFD4716_L2_2.fq.gz',
 'DJ-2-LFD4713_L2_1.fq.gz': '/media/HDD3/JC14/N2304889_30-852716646_RNA_2023-04-21/raw_data/DJ-2-LFD4713_L2_1.fq.gz',
 'OJ-3-LFD4720_L2_1.fq.gz': '/media/HDD3/JC14/N2304889_30-852716646_RNA_2023-04-21/raw_data/OJ-3-LFD4720_L2_1.fq.gz',
 'C12-1-LFD4709_L2_1.fq.gz': '/media/HDD3/JC14/N2304889_30-852716646_RNA_2023-04-21/raw_data/C12-1-LFD4709_L2_1.fq.gz',
 'OJ-1-LFD4718_L2_1.fq.gz': '/media/HDD3/JC14/N2304889_30-852716646_RNA_2023-04-21/raw_data/OJ-1-LFD4718_L2_1.fq.gz',
 'DJ-3-LFD4714_L2_2.fq.gz': '/media/HDD3/JC14/N2304889_30-852716646_RNA_2023-04-21/raw_data/DJ-3-LFD4714_L2_2.fq.gz',
 'C14-1-LFD4711_L2_2.fq.gz': '/media/HDD3/JC14/N2304889_30-852716646_RNA_2023-04-21/raw_data/C14-1-LFD4711_L2_2.fq.gz',
 'CJ-2-LFD4716_L2_1.fq.gz': '/media/HDD3/JC14/N2304889_30-852716646_RNA_2023-04-21/raw_data/CJ-2-LFD4716_L2_1.fq.gz',
 'OJ-2-LFD4719_L2_1.fq.gz': '/media/HDD3/JC14/N23048

In [5]:
list_of_sample = set([re.sub('_\d\.fq\.gz$', '', fq) for fq in list_of_fq.keys()])
print(list_of_sample)
print(len(list_of_sample))


{'OJ-3-LFD4720_L2', 'DJ-3-LFD4714_L2', 'CJ-2-LFD4716_L2', 'DJ-2-LFD4713_L2', 'C12-1-LFD4709_L2', 'C12-2-LFD4710_L2', 'CJ-3-LFD4717_L2', 'C14-1-LFD4711_L2', 'DJ-1-LFD4712_L2', 'OJ-2-LFD4719_L2', 'CJ-1-LFD4715_L2', 'OJ-1-LFD4718_L2'}
12


In [6]:
output_dir = '/media/HDD3/JC14/'
if not os.path.exists(output_dir):
    os.makedirs(output_dir)
    
sam_path = output_dir + '/1_sam/'
bam_path = output_dir + '/2_bam/'
if not os.path.exists(sam_path):
    os.makedirs(sam_path)
    
if not os.path.exists(bam_path):
    os.makedirs(bam_path)

In [7]:
Suffix_1 = '_1.fq.gz'
Suffix_2 = '_2.fq.gz'

In [8]:
hisat_index = '/media/HDD2/Genomes/Ath_Ensembl56/HGFM_'

In [9]:
def hisat2_samtool_index(i):
    
    print("hisat2",  i + '.sam', file=open(output_dir + 'hisat2_log.txt', 'a'))
    hisat2 = subprocess.run(['hisat2', '-p', '30', '--dta -x', hisat_index,
                   '-1', list_of_fq[ i + Suffix_1 ],
                   '-2', list_of_fq[ i  + Suffix_2],
                    '-S', sam_path + i +'.sam'
                   ] ,              
                   capture_output=True)
    hisat2_stdout = hisat2.stdout.decode("utf-8")
    hisat2_stderr = hisat2.stderr.decode("utf-8")
    print('for sample ', i, '\n', hisat2_stdout, hisat2_stderr )
    print('for sample ', i, '\n', hisat2_stdout, hisat2_stderr, file=open(output_dir + 'hisat2.txt', 'a'))
    print(i, '\t', 'hisat2 finished', file = open(output_dir + 'log.txt', 'a'))

    
    print(i, '\t', 'start processing ', file = open(output_dir + 'samtools_sort.txt', 'a'))

    samtools_sort = subprocess.run(['samtools', 'sort', 
                           '-@', '30', # 47 threads
                           '-o', #Write the final sorted output to FILE, rather than to standard output.
                           bam_path + i + '_sorted.bam',
                              sam_path + i +'.sam' #original .sam file
                          ],   capture_output=True)
    
    samtools_sort_stdout = samtools_sort.stdout.decode("utf-8")
    samtools_sort_stderr = samtools_sort.stderr.decode("utf-8")
    print(i, samtools_sort_stdout, samtools_sort_stderr)
    print(i, samtools_sort_stdout, samtools_sort_stderr,'\n', file = open(output_dir + 'samtools_sort.txt', 'a'))
    print(i, '\tbam finished', file = open(output_dir + 'log.txt', 'a'))
    
    print("samtool index "   + i + '_sorted.bam' + '.bai', file=open(output_dir + 'samtool_index.txt', 'a'))    
    # generate index .bai file 
    bai = subprocess.run(['samtools', 'index', '-@', '8',
                         bam_path + i + '_sorted.bam'
                          ],
                    capture_output=True)
    bai_stdout = bai.stdout.decode("utf-8")
    bai_stderr = bai.stderr.decode("utf-8")
    print(bai_stdout, bai_stderr)
    print(bai_stdout, bai_stderr, file=open(output_dir + 'samtool_index.txt', 'a'))
    print(i, '\tbai index finished', file = open(output_dir + 'log.txt', 'a'))
    
    # the sam file is too big, just keep the bam file.
    if exists(bam_path + i + '_sorted.bam.bai'):
        os.remove( sam_path + i +'.sam')

In [10]:
pool = Pool(6)
pool.map(hisat2_samtool_index, list_of_sample)
pool.close()
pool.join()

for sample  C12-1-LFD4709_L2 
  30328896 reads; of these:
  30328896 (100.00%) were paired; of these:
    15832193 (52.20%) aligned concordantly 0 times
    13731397 (45.27%) aligned concordantly exactly 1 time
    765306 (2.52%) aligned concordantly >1 times
    ----
    15832193 pairs aligned concordantly 0 times; of these:
      1241188 (7.84%) aligned discordantly 1 time
    ----
    14591005 pairs aligned 0 times concordantly or discordantly; of these:
      29182010 mates make up the pairs; of these:
        28225007 (96.72%) aligned 0 times
        809570 (2.77%) aligned exactly 1 time
        147433 (0.51%) aligned >1 times
53.47% overall alignment rate

for sample  CJ-2-LFD4716_L2 
  31187918 reads; of these:
  31187918 (100.00%) were paired; of these:
    6699100 (21.48%) aligned concordantly 0 times
    23522364 (75.42%) aligned concordantly exactly 1 time
    966454 (3.10%) aligned concordantly >1 times
    ----
    6699100 pairs aligned concordantly 0 times; of these:
    

## Feature Counts

In [ ]:
gtf_file = '/media/HDD2/Genomes/Ath_Ensembl56/Arabidopsis_thaliana.TAIR10.56.gtf'
gtf_file

'/media/HDD2/Genomes/Ath_Ensembl56/Arabidopsis_thaliana.TAIR10.56.gtf'

In [ ]:
# it will be slow if using sam files, so alwasy use bam

In [ ]:
from glob import glob
bam_dir = output_dir +  '2_bam/'
bam_files = glob(bam_dir + "*_sorted.bam")
bam_files

['/media/HDD3/JC14/2_bam/CJ-1-LFD4715_L2_sorted.bam',
 '/media/HDD3/JC14/2_bam/CJ-2-LFD4716_L2_sorted.bam',
 '/media/HDD3/JC14/2_bam/C12-2-LFD4710_L2_sorted.bam',
 '/media/HDD3/JC14/2_bam/OJ-2-LFD4719_L2_sorted.bam',
 '/media/HDD3/JC14/2_bam/DJ-3-LFD4714_L2_sorted.bam',
 '/media/HDD3/JC14/2_bam/DJ-2-LFD4713_L2_sorted.bam',
 '/media/HDD3/JC14/2_bam/OJ-3-LFD4720_L2_sorted.bam',
 '/media/HDD3/JC14/2_bam/OJ-1-LFD4718_L2_sorted.bam',
 '/media/HDD3/JC14/2_bam/C14-1-LFD4711_L2_sorted.bam',
 '/media/HDD3/JC14/2_bam/C12-1-LFD4709_L2_sorted.bam',
 '/media/HDD3/JC14/2_bam/CJ-3-LFD4717_L2_sorted.bam',
 '/media/HDD3/JC14/2_bam/DJ-1-LFD4712_L2_sorted.bam']

In [ ]:
#gene_id
temp = subprocess.run( [ 'featureCounts', 
                '-T', '64' ,
                '-p' ,
                '-t', 'exon' , #by default
                '-g', 'gene_id' ,
                '-a',  gtf_file,
                '-o', './featureCounts/JC14_all_feature_of_genes.txt',
                 
'/media/HDD3/JC14/2_bam/CJ-1-LFD4715_L2_sorted.bam',
 '/media/HDD3/JC14/2_bam/CJ-2-LFD4716_L2_sorted.bam',
 '/media/HDD3/JC14/2_bam/C12-2-LFD4710_L2_sorted.bam',
 '/media/HDD3/JC14/2_bam/OJ-2-LFD4719_L2_sorted.bam',
 '/media/HDD3/JC14/2_bam/DJ-3-LFD4714_L2_sorted.bam',
 '/media/HDD3/JC14/2_bam/DJ-2-LFD4713_L2_sorted.bam',
 '/media/HDD3/JC14/2_bam/OJ-3-LFD4720_L2_sorted.bam',
 '/media/HDD3/JC14/2_bam/OJ-1-LFD4718_L2_sorted.bam',
 '/media/HDD3/JC14/2_bam/C14-1-LFD4711_L2_sorted.bam',
 '/media/HDD3/JC14/2_bam/C12-1-LFD4709_L2_sorted.bam',
 '/media/HDD3/JC14/2_bam/CJ-3-LFD4717_L2_sorted.bam',
 '/media/HDD3/JC14/2_bam/DJ-1-LFD4712_L2_sorted.bam'

                     ],
                capture_output=True )

temp_stdout = temp.stdout.decode("utf-8")
temp_stderr = temp.stderr.decode("utf-8")
print(temp_stdout)
print(temp_stderr)
 
# -T 使用的线程数
# -p 如果是paird end 就用
# -t 将exon作为一个feature
# -g 将gene_id作为一个feature
# -a 参考的gtf/gff
# -o 输出文件
# 最后加上bam文件，有几个就加几个



        ==========     _____ _    _ ____  _____  ______          _____  
        =====         / ____| |  | |  _ \|  __ \|  ____|   /\   |  __ \ 
          =====      | (___ | |  | | |_) | |__) | |__     /  \  | |  | |
            ====      \___ \| |  | |  _ <|  _  /|  __|   / /\ \ | |  | |
              ====    ____) | |__| | |_) | | \ \| |____ / ____ \| |__| |
        ==========   |_____/ \____/|____/|_|  \_\______/_/    \_\_____/
	  v2.0.3

//========================== featureCounts setting ===========================\\
||                                                                            ||
||             Input files : 12 BAM files                                     ||
||                                                                            ||
||                           CJ-1-LFD4715_L2_sorted.bam                       ||
||                           CJ-2-LFD4716_L2_sorted.bam                       ||
||                           C12-2-LFD4710_L2_sorted.bam        

In [ ]:
#transcript_id
temp = subprocess.run( [ 'featureCounts', 
                '-T', '64' ,
                '-p' ,
                '-t', 'exon' ,
                '-g', 'transcript_id' ,
                '-a',  gtf_file,
                '-o', './featureCounts/JC14_all_feature_of_transcripts.txt',
                 
'/media/HDD3/JC14/2_bam/CJ-1-LFD4715_L2_sorted.bam',
 '/media/HDD3/JC14/2_bam/CJ-2-LFD4716_L2_sorted.bam',
 '/media/HDD3/JC14/2_bam/C12-2-LFD4710_L2_sorted.bam',
 '/media/HDD3/JC14/2_bam/OJ-2-LFD4719_L2_sorted.bam',
 '/media/HDD3/JC14/2_bam/DJ-3-LFD4714_L2_sorted.bam',
 '/media/HDD3/JC14/2_bam/DJ-2-LFD4713_L2_sorted.bam',
 '/media/HDD3/JC14/2_bam/OJ-3-LFD4720_L2_sorted.bam',
 '/media/HDD3/JC14/2_bam/OJ-1-LFD4718_L2_sorted.bam',
 '/media/HDD3/JC14/2_bam/C14-1-LFD4711_L2_sorted.bam',
 '/media/HDD3/JC14/2_bam/C12-1-LFD4709_L2_sorted.bam',
 '/media/HDD3/JC14/2_bam/CJ-3-LFD4717_L2_sorted.bam',
 '/media/HDD3/JC14/2_bam/DJ-1-LFD4712_L2_sorted.bam'
                     ],
                capture_output=True )

temp_stdout = temp.stdout.decode("utf-8")
temp_stderr = temp.stderr.decode("utf-8")
print(temp_stdout)
print(temp_stderr)
 
# -T 使用的线程数
# -p 如果是paird end 就用
# -t 将exon作为一个feature
# -g 将gene_id作为一个feature
# -a 参考的gtf/gff
# -o 输出文件
# 最后加上bam文件，有几个就加几个



        ==========     _____ _    _ ____  _____  ______          _____  
        =====         / ____| |  | |  _ \|  __ \|  ____|   /\   |  __ \ 
          =====      | (___ | |  | | |_) | |__) | |__     /  \  | |  | |
            ====      \___ \| |  | |  _ <|  _  /|  __|   / /\ \ | |  | |
              ====    ____) | |__| | |_) | | \ \| |____ / ____ \| |__| |
        ==========   |_____/ \____/|____/|_|  \_\______/_/    \_\_____/
	  v2.0.3

//========================== featureCounts setting ===========================\\
||                                                                            ||
||             Input files : 12 BAM files                                     ||
||                                                                            ||
||                           CJ-1-LFD4715_L2_sorted.bam                       ||
||                           CJ-2-LFD4716_L2_sorted.bam                       ||
||                           C12-2-LFD4710_L2_sorted.bam        